In [1]:
# Import Dependencies
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# Data Source: BulbaPedia
url = 'https://bulbapedia.bulbagarden.net/wiki/List_of_Pokémon_by_Ability'

In [3]:
# Create soup response to find tables
response=requests.get(url)
soup=BeautifulSoup(response.text,'html.parser')

In [4]:
# Find Ability tables in html
pkmn_tables=soup.find_all('table' , class_="sortable")

In [5]:
# Declare first table as dataframe
kanto_df=pd.read_html(pkmn_tables[0].prettify())[0]

In [6]:
# merge pkmn tables from all generations
all_pkmn_df=kanto_df[1:]
for gen in range(1,7):
#     trim header row to enable merge
    gen_df = pd.read_html(pkmn_tables[gen].prettify())[0][1:]
    all_pkmn_df=all_pkmn_df.append(gen_df)

In [7]:
# Declare Defensive Abilities
def_abilities=[
    'Dry Skin'
    'Filter',
    'Flash Fire',
    'Fluffy',
    'Heatproof',
    'Levitate',
    'Lightning Rod',
    'Motor Drive',
    'Prism Armor',
    'Sap Sipper',
    'Solid Rock',
    'Storm Drain',
    'Thick Fat',
    'Volt Absorb',
    'Water Absorb',
    'Water Bubble',
    'Wonder Guard',
]

In [8]:
# Remove contextual * from Pokemon and ability names
count = len(all_pkmn_df)
for form in range(count):
#     Convert national pokedex numbers to integers
    all_pkmn_df[0].iloc[form]=int(all_pkmn_df[0].iloc[form])
#     Remove contextual * from Pokemon and ability names
    for col in range(2,6):
        if type(all_pkmn_df[col].iloc[form])!=type(float()):
            if all_pkmn_df[col].iloc[form][-1]=="*":
                all_pkmn_df[col].iloc[form]=all_pkmn_df[2].iloc[col][0:-2]
#             Move relevant defensive ability to column 1
            if col > 2:
                if all_pkmn_df[col].iloc[form] in def_abilities:
                    all_pkmn_df[1].iloc[form] = all_pkmn_df[col].iloc[form]

C:\Users\OW93790\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
# Drop picture column
nat_pkmn_df=all_pkmn_df[[2,1]]
# # Show show defensive abilities of non-relevant pokemon as "N/A"
# nat_pkmn_df[1][nat_pkmn_df[1].isnull()]="N/A"
# Rename Columns
nat_pkmn_df=nat_pkmn_df.rename(columns={1:'def_abil',2:'name'})

# reindex combined table
count=len(nat_pkmn_df)
indices=[i for i in range(count)]
nat_pkmn_df.index=indices

# Add forme names to Giratina and Thundurus
nat_pkmn_df.iloc[574]['name']='Giratina (Altered Forme)'
nat_pkmn_df.iloc[575]['name']='Giratina (Origin Forme)'
nat_pkmn_df.iloc[741]['name']='Thundurus (Incarnate Forme)'
nat_pkmn_df.iloc[741]['name']='Thundurus (Therian Forme)'

# Drop multi-form pokemon without different defensive abilties or types
nat_pkmn_df=nat_pkmn_df.drop_duplicates()


nat_pkmn_df

,name,def_abil
0,Bulbasaur,NaN
1,Ivysaur,NaN
2,Venusaur,NaN
3,Mega Venusaur,Thick Fat
4,Charmander,NaN
5,Charmeleon,NaN
6,Charizard,NaN
7,Mega Charizard X,NaN
8,Mega Charizard Y,NaN
9,Squirtle,NaN


In [10]:
# Save data to csv
nat_pkmn_df.to_csv('Output/abilities.csv')

In [11]:
# Read in pokedex dataframe to merge
pokedex_df=pd.read_csv('Output/nat_dex.csv')
pokedex_df=pokedex_df.merge(nat_pkmn_df,how='left',on='name')
# drop mismatched index column
pokedex_df=pokedex_df.drop(labels=['Unnamed: 0'],axis='columns')
# drop Meltan and Melmetal because they aren't fully developed
pokedex_df=pokedex_df.drop(labels=[855,856],axis='index')

pokedex_df

,nat_dex,name,type1,type2,def_abil
0,1,Bulbasaur,Grass,Poison,NaN
1,2,Ivysaur,Grass,Poison,NaN
2,3,Venusaur,Grass,Poison,NaN
3,4,Charmander,Fire,NaN,NaN
4,5,Charmeleon,Fire,NaN,NaN
5,6,Charizard,Fire,Flying,NaN
6,7,Squirtle,Water,NaN,NaN
7,8,Wartortle,Water,NaN,NaN
8,9,Blastoise,Water,NaN,NaN
9,10,Caterpie,Bug,NaN,NaN


In [12]:
# Save data to csv
pokedex_df.to_csv('Output/pokedex_w_ability.csv')